<a href="https://colab.research.google.com/github/Dckstr6/Deep-Learning-Assignments/blob/main/FoDS_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random
from random import sample

In [3]:
dataset = pd.read_csv('/content/insurance.csv')

In [4]:
dataset['ones'] = 1
cols = ['ones','age','bmi','children','charges']
dataset = dataset[cols]
dataset.head()

,ones,age,bmi,children,charges
0,1,19,27.900,0,16884.92400
1,1,18,33.770,1,1725.55230
2,1,28,33.000,3,4449.46200
3,1,33,22.705,0,21984.47061
4,1,32,28.880,0,3866.85520


In [5]:
W_N = np.random.randint(1,10,size=(4,))

In [6]:
X = np.array(dataset[['ones','age','bmi','children']])
print(X.shape)

(1338, 4)


In [7]:
X[:,1] = (X[:,1]-np.mean(X[:,1]))/np.std(X[:,1])
X[:,2] = (X[:,2]-np.mean(X[:,2]))/np.std(X[:,2])
X[:,3] = (X[:,3]-np.mean(X[:,3]))/np.std(X[:,3])
print(X.shape)
print(X[0:5,:])

(1338, 4)
[[ 1.         -1.43876426 -0.45332    -0.90861367]
 [ 1.         -1.50996545  0.5096211  -0.07876719]
 [ 1.         -0.79795355  0.38330685  1.58092576]
 [ 1.         -0.4419476  -1.30553108 -0.90861367]
 [ 1.         -0.51314879 -0.29255641 -0.90861367]]


In [8]:
Y = np.array(dataset['charges'])
Y = (Y - np.mean(Y))/np.std(Y)
print(Y.shape)
print(Y[0:5])

(1338,)
[ 0.2985838  -0.95368917 -0.72867467  0.71984298 -0.77680218]


In [9]:
l = X.shape[0]
f = round(0.7*l)
indices = sample(range(l),f)


X_train = X[indices]
y_train = Y[indices]

X_test = np.delete(X,indices,axis=0)
y_test = np.delete(Y,indices)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(937, 4)
(937,)
(401, 4)
(401,)


## **Normal Solution Attempt**

Equation: W = (X.T * X)^-1 * X.T * Y

In [10]:
print(W_N)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

[9 8 1 8]


In [11]:
temp = np.dot((X_train.T),X_train)
temp = np.linalg.inv(temp)
temp = np.dot(temp,(X_train.T))
temp = np.dot(temp,y_train)
print(temp)
W_N = temp
print(W_N)
print(temp.shape)
print(W_N.shape)

[-0.01035712  0.29564877  0.17600499  0.0307453 ]
[-0.01035712  0.29564877  0.17600499  0.0307453 ]
(4,)
(4,)


In [12]:
def sse_error(W,X,y):
    predictions = np.dot(X,W)
    #print(predictions.shape)
    errors = np.subtract(predictions,y)
    sse = np.sum(errors**2)
    #print(sse)
    return sse

def rmse_error(W,X,y):
    predictions = np.dot(X,W)
    print(predictions.shape)
    errors = np.subtract(predictions,y)
    sse = np.sum(errors**2)
    rmse = np.sqrt(sse/predictions.shape[0])
    return rmse

In [13]:
print("SSE Errors: ")
print(sse_error(W_N,X_train,y_train))
print(sse_error(W_N,X_test,y_test))
print("RMSE Errors: ")
print(rmse_error(W_N,X_train,y_train))
print(rmse_error(W_N,X_test,y_test))

SSE Errors: 
843.2315099248962
335.4376537867687
RMSE Errors: 
(937,)
0.9486447727868715
(401,)
0.9146053122925415


In [14]:
print(temp)
print(W_N)

[-0.01035712  0.29564877  0.17600499  0.0307453 ]
[-0.01035712  0.29564877  0.17600499  0.0307453 ]


## **GRADIENT** **DESCENT**

In [15]:
W_GD = np.random.randint(1,10,size=(4,))
print(W_GD)

[2 2 2 3]


GRADIENT DESCENT UPDATE EQUATIONS:

dJ/dWj = -sigma(i)(yi - zi)xij

which means

dJ/dWj = -sigma(i)(yi - np.dot(X,W))xij

(Last product is not matrix multiplication, it is element-wise mul)

In [16]:
def prediction(W,X):
    pred = np.dot(X,W)
    return pred

In [17]:
def calc_gradient(W,X,y):
    grad = np.zeros((4,))
    diff = (y - prediction(W,X))
    grad[0] = -1*(np.sum(diff * X[:,0].reshape(X.shape[0],)))
    grad[1] = -1*(np.sum(diff * X[:,1].reshape(X.shape[0],)))
    grad[2] = -1*(np.sum(diff * X[:,2].reshape(X.shape[0],)))
    grad[3] = -1*(np.sum(diff * X[:,3].reshape(X.shape[0],)))
    #print("Gradient: ")
    #print(grad)
    return grad

In [18]:
def make_update(W,gradient,learning_rate):
    updated_W = np.array(W)
    updated_W = np.subtract(W, learning_rate*gradient)
    return updated_W

In [22]:
def gradient_descent(W,X,y,iterations,learning_rate):
    for i in range(iterations):
        print("==============================================================")
        print("Iteration : %d" % i)
        print("Current Training Error: %f" % (sse_error(W,X,y)))
        gradients = calc_gradient(W,X,y)
        W = make_update(W,gradient=gradients,learning_rate=learning_rate)
        print("Updated W: ")
        print(W)
    return W

In [23]:
W_GD = gradient_descent(W_GD,X_train,y_train,10,0.001)

Iteration : 0
Current Training Error: 937.894192
Updated W: 
[-0.00438193  0.28677209  0.19232378  0.05429111]
Iteration : 1
Current Training Error: 844.057603
Updated W: 
[-0.01047197  0.29164244  0.17772801  0.03270397]
Iteration : 2
Current Training Error: 843.250258
Updated W: 
[-0.0104575   0.29504504  0.17650857  0.03116542]
Iteration : 3
Current Training Error: 843.232160
Updated W: 
[-0.01038344  0.29552288  0.17609901  0.03081437]
Iteration : 4
Current Training Error: 843.231534
Updated W: 
[-0.0103627   0.295625    0.17602406  0.03075871]
Iteration : 5
Current Training Error: 843.231511
Updated W: 
[-0.01035825  0.29564411  0.17600869  0.03074785]
Iteration : 6
Current Training Error: 843.231510
Updated W: 
[-0.01035734  0.29564786  0.17600571  0.0307458 ]
Iteration : 7
Current Training Error: 843.231510
Updated W: 
[-0.01035717  0.29564859  0.17600513  0.0307454 ]
Iteration : 8
Current Training Error: 843.231510
Updated W: 
[-0.01035713  0.29564873  0.17600501  0.03074532]
I

In [24]:
print(W_GD)

[-0.01035712  0.29564876  0.17600499  0.03074531]


## **Stochastic Gradient Descent**

In [25]:
W_SGD =  np.random.randint(1,10,size=(4,))

def calc_stoc_grad(W, X, y):
  stocgrad=np.zeros((4,))
  i = random.randint(0,y.shape[0]-1)
  #print(i)
  pred = np.dot(X[i,:] , W)
  #print(pred)
  sdiff = y[i,] - pred
  #print(sdiff)

  stocgrad[0] = -1*(sdiff * X[i,0])
  stocgrad[1] = -1*(sdiff * X[i,1])
  stocgrad[2] = -1*(sdiff * X[i,2])
  stocgrad[3] = -1*(sdiff * X[i,3])

  return stocgrad
#calc_stoc_grad(W_SGD,X_train, y_train)


In [26]:
def update( W, sgradient , alpha):
  new_W = np.array(W)
  new_W = np.subtract(W, alpha*sgradient)
  return new_W

In [27]:
def stoc_gradient_descent(W,X,y,iterations,n):
  for i in range(iterations):
    print("########################################################")
    print("ITERATION NO: %d" % i )
    print("CURRENT TRAINING ERROR (SSE) : %f" % (sse_error(W,X,y)))
    print("CURRENT TRAINING ERROR (RMSE) : %f" % (rmse_error(W,X,y)))
    gradients=calc_stoc_grad(W,X,y)
    W = update(W, sgradient = gradients, alpha = n)
    print("NEW W: ", W)
  return W

In [28]:
W_SGD = stoc_gradient_descent(W_SGD, X_train, y_train, 10000, 0.001)

Streaming output truncated to the last 5000 lines.
NEW W:  [0.00211652 0.29876458 0.17929799 0.03806098]
########################################################
ITERATION NO: 9167
CURRENT TRAINING ERROR (SSE) : 843.453574
(937,)
CURRENT TRAINING ERROR (RMSE) : 0.948770
NEW W:  [0.00178439 0.29834381 0.17967739 0.03836276]
########################################################
ITERATION NO: 9168
CURRENT TRAINING ERROR (SSE) : 843.450295
(937,)
CURRENT TRAINING ERROR (RMSE) : 0.948768
NEW W:  [0.00109192 0.29805819 0.17933244 0.03841731]
########################################################
ITERATION NO: 9169
CURRENT TRAINING ERROR (SSE) : 843.431276
(937,)
CURRENT TRAINING ERROR (RMSE) : 0.948757
NEW W:  [0.00049118 0.29725434 0.17896519 0.03896315]
########################################################
ITERATION NO: 9170
CURRENT TRAINING ERROR (SSE) : 843.420227
(937,)
CURRENT TRAINING ERROR (RMSE) : 0.948751
NEW W:  [-0.00056925  0.29628843  0.1759946   0.03904668]
###########

In [29]:
print(W_SGD)

[-0.0014923   0.2912522   0.17911236  0.06257581]
